#Применяем обученную модель на новых скачанных фотографиях из интернета

In [ ]:
# Переменные с путями к папкам
cars_dir = "/content/drive/My Drive/cars"
masks_dir = "/content/drive/My Drive/masks"
weights_dir = "/content/drive/My Drive/ai/weights"

In [ ]:
# Подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Создаем папку куда будем сохранять сгенерированные маски
!mkdir "$masks_dir"

In [ ]:
# Загружаем веса в модель из папки с весами на гугл диске
id2label = {0: 'background', 1: 'frame', 2: 'windows', 3: 'lights', 4: 'wheels'}
label2id = {'background': 0, 'frame': 1, 'windows': 2, 'lights': 3, 'wheels': 4}
id2color = {0: [0, 0, 0], 1: [255, 255, 255], 2: [255, 0, 0], 3: [0, 255, 0], 4: [0, 0, 255]}
import torch
import os
from transformers import SegformerFeatureExtractor
feature_extractor = SegformerFeatureExtractor()

from transformers import SegformerForSemanticSegmentation

model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=5,
                                                         id2label=id2label,
                                                         label2id=label2id,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
weights_files = [f for f in os.listdir(weights_dir) if f.endswith('.pth')]
if weights_files:
    weights_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))
    latest_weights_file = weights_files[-1]
    print(f"Loading weights from {latest_weights_file}")
    if torch.cuda.is_available():
      model.load_state_dict(torch.load(os.path.join(weights_dir, latest_weights_file)))
    else:
      model.load_state_dict(torch.load(os.path.join(weights_dir, latest_weights_file), map_location=torch.device('cpu')))

In [ ]:
# Функция применения модель к картинке
from torch import nn
import numpy as np
from PIL import Image


def apply_model(image):
    image = image.convert("RGB")
    encoding = feature_extractor(image, return_tensors="pt")
    pixel_values = encoding.pixel_values.to(device)
    outputs = model(pixel_values=pixel_values)
    logits = outputs.logits.cpu()
    # Изменяем масштаб логитов до исходного размера картинки
    upsampled_logits = nn.functional.interpolate(logits,
                size=image.size[::-1], # высота, ширина
                mode='bilinear',
                align_corners=False)
    # Применяем argmax по размерности класса
    seg = upsampled_logits.argmax(dim=1)[0]
    color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # высота, ширина, 3
    for label, color in id2color.items():
        color_seg[seg == label, :] = color
    return Image.fromarray(color_seg)

И наконец применяем модель ко всем картинкам в папке cars_dir, сгенерированные маски складываем в masks_dir

In [ ]:
import os

def process_file(file_path):
    image = Image.open(file_path)
    result = apply_model(image)
    filename = os.path.basename(file_path)
    result.save(masks_dir + '/' + filename)
    print("Применяю модель к:", filename)



def iterate_directory(directory_path):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            process_file(file_path)

directory_path = cars_dir
iterate_directory(directory_path)
print("Сгенерированы маски для всех файлов")